# 01 — Preprocessing & EDA (Beijing Multi-Site Air Quality)
Mục tiêu: tải dữ liệu, làm sạch, tạo nhãn phân lớp (AQI class theo PM2.5 24h mean), tạo đặc trưng thời gian + lag, và lưu `data/processed/cleaned.parquet`.

**Lưu ý:** nếu `USE_UCIMLREPO=True` thì notebook cần internet để tải dataset từ UCI.

In [ ]:
USE_UCIMLREPO = False
RAW_ZIP_PATH = "data/raw/PRSA2017_Data_20130301-20170228.zip"

OUTPUT_CLEANED_PATH = 'data/processed/cleaned.parquet'
LAG_HOURS=[1, 3, 24]


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

from src.classification_library import (
    load_beijing_air_quality,
    clean_air_quality_df,
    add_pm25_24h_and_label,
    add_time_features,
    add_lag_features,
)

PROJECT_ROOT = Path('..').resolve()
OUT_PATH = (PROJECT_ROOT / OUTPUT_CLEANED_PATH).resolve()
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)


In [ ]:
df_raw = load_beijing_air_quality(use_ucimlrepo=USE_UCIMLREPO, raw_zip_path=RAW_ZIP_PATH)
print('raw shape:', df_raw.shape)
df_raw.head()

In [ ]:
df = clean_air_quality_df(df_raw)
df = add_pm25_24h_and_label(df)
df = add_time_features(df)
df = add_lag_features(df, lag_hours=LAG_HOURS)
print('cleaned shape:', df.shape)
df[['datetime','station','PM2.5','pm25_24h','aqi_class']].head(10)

In [ ]:
# EDA nhanh: missingness và phân bố lớp
missing_rate = df.isna().mean().sort_values(ascending=False)
missing_rate.head(20)

In [ ]:
# Kiểm tra khoảng thời gian và tính liên tục
start_date = df['datetime'].min()
end_date = df['datetime'].max()
total_hours = (end_date - start_date).total_seconds() / 3600 + 1

print(f"Khoảng thời gian: {start_date} đến {end_date}")
print(f"Số giờ dự kiến: {total_hours}")
print(f"Số dòng thực tế mỗi trạm: {len(df)/df['station'].nunique()}")

# Giải thích: PM2.5 là biến quan trọng nhất nhưng có tỷ lệ thiếu (~2.08%). 
# Nếu thiếu tập trung vào các đợt cao điểm, dự báo sẽ mất chính xác.

In [ ]:
class_dist = df['aqi_class'].value_counts(dropna=False)
class_dist

In [ ]:
import matplotlib.pyplot as plt

class_dist.drop(index=[x for x in class_dist.index if pd.isna(x)], errors='ignore').plot(kind='bar')
plt.title('AQI class distribution (PM2.5 24h mean)')
plt.ylabel('count')
plt.tight_layout()
plt.show()

In [ ]:
df.to_parquet(OUT_PATH, index=False)
print('Saved:', OUT_PATH)

In [ ]:
import matplotlib.pyplot as plt

# Chọn 1 trạm để phân tích (ví dụ: Aotizhongxin)
station_name = 'Aotizhongxin'
df_station = df[df['station'] == station_name].set_index('datetime')

# Hình 1: PM2.5 toàn giai đoạn
plt.figure(figsize=(15, 5))
plt.plot(df_station['PM2.5'])
# Xóa bỏ phần  đi
plt.title(f'Nồng độ PM2.5 toàn giai đoạn tại {station_name}')
plt.show()

# Hình 2: Zoom 1 tháng để thấy tính chu kỳ (Seasonality)
plt.figure(figsize=(15, 5))
plt.plot(df_station['PM2.5'].iloc[:720]) # 720 giờ ~ 1 tháng
plt.title(f'Nồng độ PM2.5 Zoom 1 tháng tại {station_name}')
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss
# Loại bỏ giá trị NaN trước khi kiểm định
series = df_station['PM2.5'].dropna()

# Kiểm định ADF
adf_result = adfuller(series)
print(f'ADF p-value: {adf_result[1]}') # Nếu < 0.05 => Chuỗi dừng [cite: 88, 211]

# Kiểm định KPSS
kpss_result = kpss(series)
print(f'KPSS p-value: {kpss_result[1]}') # Nếu > 0.05 => Chuỗi dừng [cite: 89, 211]

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt

# Vẽ ACF để chứng minh tính mùa vụ s=24
plt.figure(figsize=(12, 5))
plot_acf(series, lags=48) # Xem trong 48 giờ
plt.title('Biểu đồ ACF chứng minh tính mùa vụ (lặp lại tại lag 24)')
plt.show()